In [ ]:
# First, we add all the packages we need to the system path.
import os
from dotenv import load_dotenv
from contextlib import AsyncExitStack
from typing import Optional, Set

from azure.ai.agents import AgentsClient
from azure.ai.agents.models import FunctionTool,ToolSet
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from Helpers.agent_team import AgentTeam, AgentTask

In [ ]:
# Here, we connect to the Azure AI Agents service.
load_dotenv()
project_endpoint = os.getenv("PROJECT_ENDPOINT")
model_deployment = os.getenv("MODEL_DEPLOYMENT_NAME")
nonProdTenantId = os.getenv("TENANT_ID")

os.system(f"az config set core.login_experience_v2=off")
os.system(f"az login --tenant {nonProdTenantId}")

subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID_FOR_AGENTS")
print (f"Subscription ID - AZURE_SUBSCRIPTION_ID_FOR_AGENTS : {subscription_id}")

os.system(f"az account set --subscription {subscription_id}")
credential = DefaultAzureCredential()

In [ ]:
# Global variables to maintain state across cells
from mcp import ClientSession, StdioServerParameters, stdio_client


session = None
exit_stack = None

async def connect_to_server():
    global session, exit_stack

    # Clean up existing connection if any
    if exit_stack:
        await exit_stack.aclose()

    # Create new exit stack
    exit_stack = AsyncExitStack()

    try:
        server_params = StdioServerParameters(
            command="python",
            args=["server.py"],
        )

        # Start the MCP server
        stdio_transport = await exit_stack.enter_async_context(stdio_client(server_params))
        stdio, write = stdio_transport  

        # Create an MCP client session
        session = await exit_stack.enter_async_context(ClientSession(stdio, write))
        await session.initialize()

        # List available tools
        response = await session.list_tools()
        tools = response.tools
        print("\nConnected to server with tools:", [tool.name for tool in tools]) 

        return session, tools
    except Exception as e:
        print(f"Error connecting to server: {e}")
        if exit_stack:
            await exit_stack.aclose()
        raise

# Helper function to clean up resources
async def cleanup_connection():
    global exit_stack
    if exit_stack:
        await exit_stack.aclose()
        exit_stack = None
        print("Connection cleaned up")

In [ ]:
def create_task(team_name: str, recipient: str, request: str, requestor: str) -> str:
    """
    Requests another agent in the team to complete a task.

    :param team_name (str): The name of the team.
    :param recipient (str): The name of the agent that is being requested to complete the task.
    :param request (str): A description of the to complete. This can also be a question.
    :param requestor (str): The name of the agent who is requesting the task.
    :return: True if the task was successfully received, False otherwise.
    :rtype: str
    """
    task = AgentTask(recipient=recipient, task_description=request, requestor=requestor)
    team: Optional[AgentTeam] = None
    try:
        team = AgentTeam.get_team(team_name)
    except:
        pass
    if team is not None:
        team.add_task(task)
        return "True"
    return "False"

In [ ]:
project_client = AIProjectClient(
    endpoint=os.environ["PROJECT_ENDPOINT"],
    credential=DefaultAzureCredential(),
)

# Build a function for each tool
def make_tool_func(tool_name):
    async def tool_func(**kwargs):
        result = await session.call_tool(tool_name, kwargs)
        return result
    
    tool_func.__name__ = tool_name
    return tool_func

# Any additional functions that might be used by the agents:
# Combine all functions into a single FunctionTool
agent_team_default_functions: Set = {
    create_task,
}

default_function_tool = FunctionTool(functions=agent_team_default_functions)
session, tools = await connect_to_server()
functions_dict = {tool.name: make_tool_func(tool.name) for tool in tools}

agents_client = project_client.agents
agents_client.enable_auto_function_calls({create_task})

toolset = ToolSet()
toolset.add(default_function_tool)  # Add only the default function tool


agent_team = AgentTeam("Inventory Management Team", agents_client=agents_client)
agent_team.set_team_leader(
    model=model_deployment,
    name="TeamLeader",
    instructions="You are an agent named 'TeamLeader'. You are a leader of a team of agents. The name of your team is 'Inventory Management Team'."
    "You are an agent that is responsible for receiving requests from the user and utilizing a team of agents to complete the task. "
    "When you are passed a request, the only thing you will do is evaluate which team member should do which task next to complete the request. "
    "You will use the provided create_task function to create a task for the agent that is best suited for handling the task next. "
    "You will respond with the description of who you assigned the task and why. When you think that the original user request is "
    "processed completely utilizing all the talent available in the team, you do not have to create anymore tasks. "
    "Using the skills of all the team members when applicable is highly valued. "
    "Do not create parallel tasks. "
    "Here are the other agents in your team: "
    "- InventoryAgent: You are an inventory management agent. You use tools to look up items in the inventory, and provide inventory suggestions. Your name is InventoryAgent. "
    "- AccountingAgent: You are an accounting agent. You take suggestions from the inventory management agent on what needs to happen, and any increase in Units In Stock combined must not exceed $1000. Your name is AccountingAgent.",
    toolset=toolset,
)
agent_team.add_agent(
    model=model_deployment,
    name="InventoryAgent",
    instructions="You are an inventory management agent. You use tools to look up items in the inventory, and provide inventory suggestions. Your name is InventoryAgent.",
)

agent_team.add_agent(
    model=model_deployment,
    name="AccountingAgent",
    instructions="You are an accounting agent. You take suggestions from the inventory management agent on what needs to happen, and any increase in Units In Stock combined must not exceed $1000. Your name is AccountingAgent.",
)

agent_team.assemble_team()

print("Team assembled successfully.")

#agent_team.dismantle_team()

In [ ]:
user_input = input("Can you give me an overview of the inventory? ")

agent_team.process_request(request=user_input)

In [ ]:
agent_team.dismantle_team()